In [1]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [34]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [35]:
train = pd.read_csv("latest_cleaned_data/train_lemmatized_withstop.csv")
train = train[~train['user_review'].isna()]
train = train[train['user_review'] != ' ']

In [36]:
user_review = train.user_review.values

In [37]:
count = 0
corpus_vec = np.zeros((len(user_review),300))
docs = nlp.pipe(user_review, disable=['ner','tagger','parser'])
pbar = tqdm(total = user_review.shape[0])
for doc in docs:
    sent_vec = []
    for token in doc:
        if token.has_vector:
            sent_vec.append(list(token.vector))
    sent_vec = np.array(sent_vec)
    corpus_vec[count] = np.mean(sent_vec,axis=0)
    if count%1000==0:
        pbar.update(1000)
    count+= 1
pbar.close()

18000it [01:54, 156.58it/s]                           


In [38]:
len(corpus_vec)

17471

In [40]:
D = np.argwhere(np.isnan(corpus_vec))
idx = list(set(D[:,0]))
train.iloc[idx,:]

,review_id,title,year,user_review,user_suggestion
3980,4444,Team Fortress 2,2018.0,pootis pootis pootis pootis pootis pootis poot...,1
16701,24670,Bloons TD Battles,2017.0,poppoppoppoppoppoppoppoppoppoppoppoppoppoppopp...,1
12411,17908,RaceRoom Racing Experience,2014.0,vroooooooooooooooooooooooooooooooooooooooooooo...,1
77,78,Spooky's Jump Scare Mansion,2017.0,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,1
9600,13184,Realm of the Mad God,2014.0,yessssssssssssssssssssssssssssssssssssssssssss...,1
9444,13028,Realm of the Mad God,2013.0,gooooooooooooooooooooooooooooooooooooooooooooo...,1
16747,24716,Bloons TD Battles,2018.0,baaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,0
12617,18114,RaceRoom Racing Experience,2014.0,freefreefreefreefreefreefreefreefreefreefreefr...,1
449,450,Sakura Clicker,2017.0,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,1


In [42]:
print(train.shape)
print(corpus_vec.shape)

(17471, 5)
(17471, 300)


In [43]:
train = train.drop(index=idx) #Deleting from dataframe
corpus_vec = np.delete(corpus_vec,idx,axis=0) #Deleting embeddings

In [44]:
print(train.shape)
print(corpus_vec.shape)

(17462, 5)
(17462, 300)


## Embeddings Generated Machine Learning

In [59]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [60]:
X_train,X_test,y_train,y_test = train_test_split(corpus_vec,train.user_suggestion.values,stratify=train.user_suggestion.values,test_size=0.3)
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((12223, 300), (12223,), (5239, 300), (5239,))

In [64]:
lr = LogisticRegression(max_iter=500)
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
print(f1_score(y_test,y_pred))
confusion_matrix(y_test,y_pred)

0.8409090909090909


array([[1720,  535],
       [ 431, 2553]])

## Summation of Embeddings